In [1]:
import calibrate
import simulate
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from time import time

In [121]:
mu, alpha, beta = 0.5, 0.2, 0.3
hwk_proc = simulate.HawkesProcess(mu, alpha, beta)
T = 10
events = hwk_proc.simulate(T)
intensity_func = lambda t: hwk_proc.get_rate(events, t)

# plot the events
# plt.figure()
# plt.plot(events, [0]*len(events), 'x')
# t_range = np.linspace(0, T, 1000)
# intensity = [intensity_func(t) for t in t_range]
# plt.plot(t_range, intensity)
# plt.show()

class HawkesProcessCalibrator:
    def __init__(self, events):
        self.events = events

    def calibrate(self, timeit=False):
        # source: https://www.ism.ac.jp/editsec/aism/pdf/031_1_0145.pdf
        def nll(theta):
            mu, alpha, beta = theta
            t_n = self.events[-1]
            n = len(self.events)
            ll = - mu * t_n
            for i in range(n):
                ll += alpha/beta * (np.exp(-beta*(t_n - self.events[i])) - 1)
            for i in range(n):
                A_i = sum(np.exp(-beta*(self.events[i] - self.events[j])) for j in range(i))
                ll += np.log(mu + alpha*A_i)
            return -ll
        
        start = time()
        res = minimize(fun=nll, x0=[1,1,2], bounds=[(1e-6, None), (1e-6, None), (1e-6, None)], method='L-BFGS-B')
        end = time()
        if timeit:
            print("\033[92mCalibration time: ", time() - start, "s\033[0m")
        return res.x
        

cal = HawkesProcessCalibrator(events)
mu, alpha, beta = cal.calibrate()
print("Calibrated mu, alpha, beta: ", mu, alpha, beta)
print("True mu, alpha, beta: ", hwk_proc.mu, hwk_proc.alpha, hwk_proc.beta)

Calibrated mu, alpha, beta:  0.37680808143291944 1e-06 2.3242139218561317
True mu, alpha, beta:  0.5 0.2 0.3
